In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time

In [2]:
data = pd.read_csv("cattle_data_train.csv")

features = data.iloc[:, 1:-1]
yields = data.iloc[:, -1]



In [ ]:
# Feature Removal and Preprocessing
# Based on correlation analysis and data quality issues

# Features to remove (15 total):
features_to_remove = [
    'Feed_Quantity_lb',      # Duplicate of Feed_Quantity_kg (99.99% correlation)
    'Cattle_ID',             # Unique identifier, no predictive value
    'Rumination_Time_hrs',   # 55% negative values - data quality issue
    'HS_Vaccine',            # Very low correlation (0.000034)
    'BQ_Vaccine',            # Very low correlation (0.000466)
    'BVD_Vaccine',           # Very low correlation (0.000491)
    'Brucellosis_Vaccine',   # Very low correlation (0.002089)
    'FMD_Vaccine',           # Very low correlation (0.002477)
    'Resting_Hours',         # Nearly zero correlation (0.001653)
    'Housing_Score',         # Low correlation (0.004) + 3% missing values
    'Feeding_Frequency',     # No correlation (0.000380)
    'Walking_Distance_km',   # No correlation (0.001538)
    'Body_Condition_Score',  # No correlation (0.001647)
    'Humidity_percent',      # Very low correlation (0.002153)
    'Grazing_Duration_hrs',  # Very low correlation (0.004350)
    'Milking_Interval_hrs'   # Very low correlation (0.014734)
]

# Remove features
data_cleaned = data.drop(columns=features_to_remove)

print(f"Original shape: {data.shape}")
print(f"Cleaned shape: {data_cleaned.shape}")
print(f"Removed {len(features_to_remove)} features")

In [ ]:
# Extract Season from Date column
# Analysis shows seasons have strong effect on milk yield:
#   - Spring: 16.59 L (+6.4% vs average) - BEST season
#   - Winter: 16.12 L (+3.4% vs average)
#   - Fall:   15.70 L (+0.7% vs average)
#   - Summer: 13.94 L (-10.6% vs average) - WORST season (heat stress)
#   - Range: 2.65 L difference between best and worst seasons!

# Convert Date to datetime
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'])

# Extract month temporarily to create seasons
data_cleaned['Month'] = data_cleaned['Date'].dt.month

# Create Season feature (meteorological seasons)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # 9, 10, 11
        return 'Fall'

data_cleaned['Season'] = data_cleaned['Month'].apply(get_season)

# Drop both Date and Month (we only keep Season)
data_cleaned = data_cleaned.drop(columns=['Date', 'Month'])

print("Replaced Date with Season:")
print("  - Season (Winter/Spring/Summer/Fall)")
print(f"\nSeason distribution:")
print(data_cleaned['Season'].value_counts().sort_index())
print(f"\nFinal shape: {data_cleaned.shape}")

In [ ]:
# Update features and target using cleaned data
features = data_cleaned.drop(columns=['Milk_Yield_L'])
yields = data_cleaned['Milk_Yield_L']

print(f"Features shape: {features.shape}")
print(f"Target shape: {yields.shape}")

## Summary of Feature Selection

**Removed 16 features:**
1. Feed_Quantity_lb - duplicate of Feed_Quantity_kg (99.99% correlation)
2. Cattle_ID - unique identifier, no predictive value
3. Rumination_Time_hrs - data quality issue (55% negative values)
4-8. Low-correlation vaccines: HS, BQ, BVD, Brucellosis, FMD
9-15. Zero/near-zero correlation: Resting_Hours, Housing_Score, Feeding_Frequency, Walking_Distance_km, Body_Condition_Score, Humidity_percent, Grazing_Duration_hrs
16. Milking_Interval_hrs - very low correlation (0.015)

**Replaced Date with Season:**
- Removed: Date (raw timestamp)
- Added: Season (Winter/Spring/Summer/Fall)
- Rationale: Strong seasonal effect on milk yield (Spring: 16.59L vs Summer: 13.94L = 2.65L range)
- Month was NOT kept (redundant with Season - only 0.1L variation within seasons)

**Final: 19 features (down from 35 = 46% reduction)**

**Categorical (7):**
- Breed, Climate_Zone, Management_System, Lactation_Stage, Feed_Type, Farm_ID, Season

**Numeric (12):**
- Age_Months (corr: 0.31), Weight_kg (0.30), Parity (0.24), Days_in_Milk (0.06), Feed_Quantity_kg (0.22), Water_Intake_L (0.12), Ambient_Temperature_C (0.04), Anthrax_Vaccine (0.07), IBR_Vaccine (0.07), Rabies_Vaccine (0.07), Previous_Week_Avg_Yield (0.09), Mastitis (0.12)

**Why this works:**
- Removed noisy, low-correlation features
- Kept strong predictors (Age, Weight, Parity, Feed)
- Captured seasonal patterns without overfitting to specific months
- Cleaner data (fixed Rumination_Time_hrs corruption)

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer

# Update categorical columns from cleaned data
cat_cols = features.select_dtypes(include=["object", "string"]).columns.tolist()
print(f"Categorical columns: {cat_cols}")

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None, mode="freq", m=5):
        self.cols = cols
        self.mode = mode
        self.m = m

    def fit(self, X, y=None):
        X = X.copy()
        self.maps = {}

        for col in self.cols:
            freq = X[col].value_counts()
            total = len(X)

            if self.mode == "freq":
                enc = freq / total
            elif self.mode == "count":
                enc = freq
            elif self.mode == "logfreq":
                enc = np.log1p(freq / total)
            elif self.mode == "smooth":
                prior = freq.sum() / total
                enc = (freq + self.m * prior) / (freq.sum() + self.m)
            else:
                raise ValueError("Unknown mode: " + self.mode)

            self.maps[col] = enc

        return self

    def transform(self, X):
        X = X.copy()
        for col in self.cols:
            X[col] = X[col].map(self.maps[col]).fillna(0)
        return X